In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
COMPETION_DATA_DIR = '/content/drive/MyDrive/input/kaggle/feedback-prize-english-language-learning'
iter_path = COMPETION_DATA_DIR + '/iterativestratification'

In [ ]:
!pip install tokenizers
!pip install transformers
!pip install iterstrat
!pip install sentencepiece

In [ ]:
pip install --upgrade pip

In [ ]:
import numpy as np
import pandas as pd
import os
import os
import gc
import re
import ast
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
sys.path.append(iter_path)
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils import checkpoint
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true
%matplotlib inline
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import sentencepiece

In [ ]:
class CFG:
    eda = False
    debug = False
    load_bert_data = False
    load_test_only = False
    model_train = False
    train_multi = True
    train_staking = False
    train_ensumbling = True
    headrow = 3
    num_workers=4
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=25
    train=True
    
if CFG.debug:
    CFG.n_fold = 5
    
if CFG.train_staking == False & CFG.train_ensumbling == False:
    CFG.train_ensumbling = True

In [ ]:
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]

In [ ]:
# def get_logger(filename=OUTPUT_DIR + 'train'):
#     from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
#     logger = getLogger(__name__)
#     logger.setLevel(INFO)
#     handler1 = StreamHandler()
#     handler1.setFormatter(Formatter("%(message)s"))
#     handler2 = FileHandler(filename=f"{filename}.log")
#     handler2.setFormatter(Formatter("%(message)s"))
#     logger.addHandler(handler1)
#     logger.addHandler(handler2)
#     return logger


# LOGGER = get_logger()


def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=42)


def get_essay(essay_id, is_train=True):
    parent_path = INPUT_DIR + 'train' if is_train else INPUT_DIR + 'test'

    try:
        essay_path = os.path.join(parent_path, f"{essay_id}.txt")
        essay_text = open(essay_path, 'r').read()
        return essay_text
    except:
        return ''

def criterion_val(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)


def criterion_train(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

In [ ]:
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [ ]:
BATCH_SIZE = 4

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=MAX_LEN,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens



In [ ]:
tokenizer = None
MAX_LEN = 640

def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)

        
    return all_train_text_feats

In [ ]:
INPUT_DIR = '/content/drive/MyDrive/input/kaggle/feedback-prize-effectiveness/'

In [ ]:
train = pd.read_csv(INPUT_DIR + 'train.csv')
if CFG.debug:
    train = train[:1000]
test = pd.read_csv(INPUT_DIR + 'test.csv')
print(train.head())
print(train.shape)
print(test.head())
print(test.shape)

In [ ]:
pseudo_train = train['essay_id'].drop_duplicates().reset_index(drop = False)

In [ ]:
pseudo_train['full_text'] = pseudo_train['essay_id'].apply(lambda x: get_essay(x, is_train=True))

In [ ]:
pseudo_train.head()

In [ ]:
!pip install readability

In [ ]:
import readability
import pandas as pd
import re
import seaborn as sns

In [ ]:
def data_cleaner(text):
    text = text.strip()
    text = re.sub(r'\n', '', text)
    text = text.lower()
    return text

In [ ]:
def calc_readability(text):

    results = readability.getmeasures(text, lang='en')

    return results['readability grades']['ARI']

In [ ]:
pseudo_train['full_text'] = pseudo_train['full_text'].apply(data_cleaner)
pseudo_train['readability'] = pseudo_train['full_text'].apply(calc_readability)
pseudo_train['words'] = pseudo_train['full_text'].apply(lambda x: len(x.split()))

In [ ]:
pseudo_train.head()

In [ ]:
sns.pairplot(pseudo_train)

In [ ]:
min_readability = 92.793281
max_readability = 396.656655
min_words = 185.000000
max_words = 783.000000

In [ ]:
pseudo_train = pseudo_train[
    (pseudo_train['readability'] >= min_readability)
    &(pseudo_train['readability'] <= max_readability)
    &(pseudo_train['words'] >= min_words)
    &(pseudo_train['words'] <= max_words)
]

In [ ]:
pseudo_train.shape

In [ ]:
ds_tr = EmbedDataset(pseudo_train)
embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

In [ ]:
MODEL_NM = COMPETION_DATA_DIR + '/huggingface-deberta-variants/deberta-base/deberta-base'
pseudo_train_text_feats = get_embeddings(MODEL_NM)

In [ ]:
MODEL_NM =  COMPETION_DATA_DIR + '/deberta-v3-large/deberta-v3-large'
pseudo_train_text_feats2 = get_embeddings(MODEL_NM)

In [ ]:
MODEL_NM = COMPETION_DATA_DIR +  '/huggingface-deberta-variants/deberta-large/deberta-large'
pseudo_train_text_feats3 = get_embeddings(MODEL_NM)

In [ ]:
MODEL_NM = COMPETION_DATA_DIR + '/huggingface-deberta-variants/deberta-large-mnli/deberta-large-mnli'
pseudo_train_text_feats4 = get_embeddings(MODEL_NM, MAX=512)

In [ ]:
MODEL_NM = COMPETION_DATA_DIR + '/huggingface-deberta-variants/deberta-xlarge/deberta-xlarge'
pseudo_train_text_feats5 = get_embeddings(MODEL_NM, MAX=512)

In [ ]:
pseudo_train_text_feats = np.concatenate([
    pseudo_train_text_feats,
    pseudo_train_text_feats2,
    pseudo_train_text_feats3,
    pseudo_train_text_feats4,
    pseudo_train_text_feats5
],
    axis=1
)

del pseudo_train_text_feats2
del pseudo_train_text_feats3
del pseudo_train_text_feats4
del pseudo_train_text_feats5
gc.collect()

print('Our concatenated embeddings have shape', pseudo_train_text_feats.shape )

In [ ]:
# all_train_text_feats = np.load(npy_path + '/all_train_text_feats.npy')

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.svm import SVR

In [ ]:
if CFG.train_multi == True:
    models = {
        'SVR(C=1)':SVR(C=1), 
        'Ridge(alpha=1)':Ridge(alpha=1), 
        # 'Lasso(alpha=1)':Lasso(alpha=1), 
        'RFR(max_depth=5, n_estimators=10)':RFR(max_depth=5, n_estimators=10)
    }
else:
    models = {
        'Ridge(alpha=1)':Ridge(alpha=1), 
    }

In [ ]:
FOLDS = CFG.n_fold

In [ ]:
np.save('/content/drive/MyDrive/input/kaggle/feedback-prize-effectiveness/pseudo_train_text_feats', pseudo_train_text_feats)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = []
scores = []
stacks = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

for model in models.keys():

    #for fold in tqdm(range(FOLDS),total=FOLDS):
    for fold in range(FOLDS):
        print('#'*25)
        print('### Fold',fold)
        print('#'*25)
        
        # dftr_ = dftr[dftr["FOLD"]!=fold]
        # dfev_ = dftr[dftr["FOLD"]==fold]
        
        # tr_text_feats = all_train_text_feats[list(dftr_.index),:]
        # ev_text_feats = all_train_text_feats[list(dfev_.index),:]
        
        # ev_preds = np.zeros((len(ev_text_feats),6))
        test_preds = np.zeros((len(pseudo_train_text_feats),6))
        # stack_preds = np.zeros((len(all_train_text_feats[list(dftr.index),:]),6))
        
        for i,t in enumerate(target_cols):

            if CFG.model_train == True:
                pass
                # print(t,', ',end='')
                # clf = models[model]
                # clf.fit(tr_text_feats, dftr_[t].values)
                # ev_preds[:,i] = clf.predict(ev_text_feats)
                # test_preds[:,i] = clf.predict(te_text_feats)
                # stack_preds[:,i] = clf.predict(all_train_text_feats[list(dftr.index),:])

                # # モデルを保存する
                # if 'google.colab' in sys.modules:
                #     filename = f'{COMPETION_DATA_DIR}/models/finalized_{model}_{t}_{fold}.sav'
                # elif 'kaggle_web_client' in sys.modules:
                #     filename = f'finalized_{model}_{t}_{fold}.sav'
                # pickle.dump(clf, open(filename, 'wb'))

            else:
     
                filename = f'{COMPETION_DATA_DIR}/models/finalized_{model}_{t}_{fold}.sav'
                clf = pickle.load(open(filename, 'rb'))
                # ev_preds[:,i] = clf.predict(ev_text_feats)
                test_preds[:,i] = clf.predict(pseudo_train_text_feats)
                # stack_preds[:,i] = clf.predict(all_train_text_feats[list(dftr.index),:])
            
            gc.collect()

        print()
        # score = comp_score(dfev_[target_cols].values,ev_preds)
        # scores.append(score)
        
        # stacks.append(stack_preds)
        
        # print("Fold : {} RSME score: {}".format(fold,score))
        preds.append(test_preds)
    
print('#'*25)
print('Overall CV RSME =',np.mean(scores))

In [ ]:
if CFG.train_ensumbling == True:
    sub = dfte.copy()

    for i,t in enumerate(target_cols): 

        ensumbe_final_pred = pd.DataFrame()
        ensumbe_final_pred_mean = pd.DataFrame()

        for ii in range(len(stacks)):
            ensumbe_final_pred = pd.concat([ensumbe_final_pred, pd.DataFrame(preds[ii], columns = target_cols)[t]], axis = 1)

        for jj in range(1, len(models)+1):
            ensumbe_final_pred_mean = pd.concat([ensumbe_final_pred_mean, ensumbe_final_pred.iloc[:, CFG.n_fold * (jj-1): CFG.n_fold * (jj)].mean(axis = 1)], axis = 1)

        if ensumbe_final_pred_mean.shape[1] >= 2:
            sub[t] = np.mean(ensumbe_final_pred_mean, axis=1)
        else:
            sub[t] = ensumbe_final_pred_mean

    sub_columns = pd.read_csv(sub_path).columns
    sub = sub[sub_columns]
    sub.head()